# Interpolacja 

### Zadanie 1
Stablicuj następujące funkcje: sqrt(x), sin(x), x^3+2x w czterech punktach należących do przedziału 0 do 10.

In [212]:
import numpy as np

def lookupTab(fx, steps, x_start = 0, x_end = 10):
    array_x = np.linspace(x_start, x_end, steps)
    array_y = np.zeros([steps])
    for i in range(len(array_x)):
        array_y[i]=fx(array_x[i])
    return list(zip(array_x, array_y))

lookup_sqrt = lookupTab(lambda x : np.sqrt(x), 4)
print(lookup_sqrt)

lookup_sin = lookupTab(lambda x : np.sin(x), 4)
print(lookup_sin)

lookup_fun = lookupTab(lambda x : x**3+2*x, 4)
print(lookup_fun)




[(0.0, 0.0), (3.3333333333333335, 1.8257418583505538), (6.666666666666667, 2.581988897471611), (10.0, 3.1622776601683795)]
[(0.0, 0.0), (3.3333333333333335, -0.19056796287548539), (6.666666666666667, 0.37415123057121996), (10.0, -0.5440211108893699)]
[(0.0, 0.0), (3.3333333333333335, 43.70370370370371), (6.666666666666667, 309.6296296296297), (10.0, 1020.0)]


### Zadanie 2 
Napisz algorytm znajdujący wielomian interpolujący Lagrange dla powyższych stablicowanych funkcji.

In [216]:
import scipy
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial

def interpolate(inpts):
    n = len(inpts)
    thepoly = n*[0]
    for i in range(len(inpts)):
        prod = 1
        # Compute Prod_{j != i} (x_i - x_j)
        for j in (j for j in range(n) if (j != i)):
            prod *= (inpts[i][0] - inpts[j][0])
        # Compute y_i/Prod_{j != i} (x_i - x_j)
        prod = inpts[i][1]/prod
        theterm = [prod] + (n-1)*[0]
        # Compute theterm := prod*Prod_{j != i} (x - x_j)
        for j in (j for j in range(n) if (j != i)):
            for k in range(n-1,0,-1):
                theterm[k] += theterm[k-1]
                theterm[k-1] *= (-inpts[j][0])
        # thepoly += theterm
        for j in range(n):
            thepoly[j] += theterm[j]
    return(thepoly)

x = np.array(np.linspace(0,10,4))

print("Sqrt(x):")
Array1=lookupTab(lambda x: np.sqrt(x),4)
print("My method:", interpolate(Array1))

#Comparison with predefined function from scipy
y = np.sqrt(x)
poly = lagrange(x, y)
print("Predefined metthod from scipy:", Polynomial(poly).coef)

print("Sin(x):")
Array2=lookupTab(lambda x: np.sin(x),4)
print("My method:", interpolate(Array2))

#Comparison with predefined function from scipy
y = np.sin(x)
poly = lagrange(x, y)
print("Predefined metthod from scipy:", Polynomial(poly).coef)

print("x^3+2x:")
Array3=lookupTab(lambda x: x**3+2*x,4)
print("My method:", interpolate(Array3))

#Comparison with predefined function from scipy
y = x**3+2*x
poly = lagrange(x, y)
print("Predefined metthod from scipy:", Polynomial(poly).coef)




Sqrt(x):
My method: [0.0, 0.7975004346701113, -0.08833641129156172, 0.004020914442623438]
Predefined metthod from scipy: [ 0.00402091 -0.08833641  0.79750043  0.        ]
Sin(x):
My method: [0.0, -0.39428133143392285, 0.13470596313982547, -0.010071804110532687]
Predefined metthod from scipy: [-0.0100718   0.13470596 -0.39428133  0.        ]
x^3+2x:
My method: [0.0, 2.000000000000014, 0.0, 1.0]
Predefined metthod from scipy: [ 1.00000000e+00 -7.10542736e-15  2.00000000e+00  0.00000000e+00]


## **Zadanie 3**

Porównaj wartość dokładną z wynikiem interpolacji dla punktów znajdujących się pomiędzy węzłami wielomianu (w połowie odległości) interpolującego. Oszacuj dokładność interpolacji. 


In [217]:
#problem z sinusem
#a0 + a1x + a2x2 + ... + an-1xn-1
import statistics

def fillArray(steps):
    arr1=np.linspace(0, 10, steps)
    arr2=np.zeros([len(arr1)-1])
    
    for i in range(len(arr1)-1):
        arr2[i]=(arr1[i+1]-arr1[i])/2+arr1[i]
    return arr2

def interpolationResults(fx, steps):
    demi_points=fillArray(steps)
    lookupTabGivenFx=lookupTab(fx, steps)
    results=np.zeros([len(demi_points)])
    
    print("Interpolation results:")
    for i in range(len(demi_points)):
        #print(LagrangeInterp(lookupTabGivenFx, demi_points[i]))
        #results[i]=LagrangeInterp(lookupTabGivenFx, demi_points[i])
        coef=interpolate(lookupTabGivenFx)
        for j in range(len(coef)):
            results[i]+=coef[j]*(demi_points[i]**j)
        print(results[i])
    return results
        
        
def normalResults(fx, steps):
    demi_points=fillArray(steps)
    y=fx
    results=np.zeros([len(demi_points)])
    print("Normal results:")
    for i in range(len(demi_points)):
        print(y(demi_points[i]))
        results[i]=y(demi_points[i])
        
    return results

def calculatePrecision(fx, steps):
    normal_results = normalResults(fx, steps)
    interpolation_results = interpolationResults(fx, steps)
    differences=np.zeros([len(normal_results)])
     
    for i in range(len(differences)):
        differences[i]=abs(interpolation_results[i]-normal_results[i])
    print("Mean difference:", statistics.mean(differences))
    print("Median difference:",statistics.median(differences) )

print("Precision for function sqrt(x)")
calculatePrecision(lambda x: np.sqrt(x),4)

print("Precision for function sin(x)")
calculatePrecision(lambda x: np.sin(x),4)

print("Precision for function x^3+2x)")
calculatePrecision(lambda x: x**3+2*x, 4)


# Instead of interpolate function
#Calculate value at given x
def LagrangeInterp(Array, x):
    unzipped = list(zip(*Array))
    Xi = unzipped[0]
    Yi = unzipped[1]
    n = len(Xi)
    result = 0
    for i in range(n):
        x_val=Xi[i]
        y_val=Yi[i]
        l = 1.0
        for k in range(n):
            if k!=i:
                l=(l*(x-Xi[k]))/(x_val-Xi[k])
               
        result+=y_val*l
    return result

Precision for function sqrt(x)
Normal results:
1.2909944487358056
2.23606797749979
2.886751345948129
Interpolation results:
1.1024038155042892
2.2817061963894436
2.838282029447705
Mean difference: 0.09423272287386471
Median difference: 0.048469316500423965
Precision for function sin(x)
Normal results:
0.9954079577517649
-0.9589242746631385
0.8872941080946946
Interpolation results:
-0.3295810441798594
0.1372669075094366
0.2403126699061815
Mean difference: 1.0227205407642375
Median difference: 1.096191182172575
Precision for function x^3+2x)
Normal results:
7.962962962962964
135.0
595.3703703703704
Interpolation results:
7.962962962962988
135.00000000000006
595.3703703703706
Mean difference: 6.483702463810914e-14
Median difference: 5.684341886080802e-14


### Zadanie 4
Powtórz powyższe kroki dla 3, 5 i 8 węzłów interpolacji - podsumuj badania. 

In [220]:
print("3 interpolation nodes:\n")
print("Precision for function sqrt(x)")
calculatePrecision(lambda x: np.sqrt(x),3)

print("Precision for function sin(x)")
calculatePrecision(lambda x: np.sin(x),3)

print("Precision for function x^3+2x)")
calculatePrecision(lambda x: x**3+2*x, 3)

print("\n\n5 interpolation nodes:\n")
print("Precision for function sqrt(x)")
calculatePrecision(lambda x: np.sqrt(x),5)

print("Precision for function sin(x)")
calculatePrecision(lambda x: np.sin(x),5)

print("Precision for function x^3+2x)")
calculatePrecision(lambda x: x**3+2*x, 5)

print("\n\n8 interpolation nodes:\n")
print("Precision for function sqrt(x)")
calculatePrecision(lambda x: np.sqrt(x),8)

print("Precision for function sin(x)")
calculatePrecision(lambda x: np.sin(x),8)

print("Precision for function x^3+2x)")
calculatePrecision(lambda x: x**3+2*x, 8)

3 interpolation nodes:

Precision for function sqrt(x)
Normal results:
1.5811388300841898
2.7386127875258306
Interpolation results:
1.2817662756037946
2.862905105687984
Mean difference: 0.21183243632127435
Median difference: 0.21183243632127435
Precision for function sin(x)
Normal results:
0.5984721441039564
0.9379999767747389
Interpolation results:
-0.6511905671361826
-0.9232011225808678
Mean difference: 1.555431905297873
Median difference: 1.555431905297873
Precision for function x^3+2x)
Normal results:
20.625
436.875
Interpolation results:
-26.250000000000014
483.7499999999999
Mean difference: 46.87499999999995
Median difference: 46.87499999999995


5 interpolation nodes:

Precision for function sqrt(x)
Normal results:
1.118033988749895
1.9364916731037085
2.5
2.958039891549808
Interpolation results:
0.9820659963803334
1.9596017578907878
2.4853806275312857
2.9830674779443713
Mean difference: 0.04968125900497958
Median difference: 0.02406883559082129
Precision for function sin(x)
Norm